# Todos
- [x] Change objective function to minimize the average deivation between actual and ideal assignment frequency between within the assignment pool of each task 
- [x] Update/Record assignment frequency csv
- [x] Assignments for service, weekly, and monthly assginment slots
- [ ] Output for html render
- [ ] Weight assignment favorability (scalar per man to augment assignment frequency)
- [ ] Validate inputs
- [ ] Some tests
- [ ] Update README
- [ ] cron


In [1]:
import calendar
calendar.setfirstweekday(calendar.SUNDAY)

import os
import re

import pandas as pd
from itertools import zip_longest
from collections import OrderedDict
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value
from roster import Roster
from schedule import Schedule
from helpers import *
from render import *
from IPython.core.display import display, HTML


/var/folders/4z/l87fr4g16qb0rxtm42zcxtym0000gn/T/ipykernel_80606/2386876518.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
schedule = Schedule(2024, 8, Roster('data/previous-assignments-ipynb.csv'))
roster = schedule.roster

people = roster.people
tasks = roster.tasks
is_eligible = roster.is_eligible
get_eligible = roster.get_eligible
is_excluded = roster.is_excluded
ideal_avg = roster.ideal_avg

all_date_tasks = schedule.all_date_tasks
get_date_tasks = schedule.get_date_tasks


In [28]:
assert trim_task_name('song_leader-0') == 'song_leader'
assert trim_task_name('song_leader-30') == 'song_leader'
assert trim_task_name('first_opening_prayer-7') == 'first_opening_prayer'
assert trim_task_name('sound_board_operator') == 'sound_board_operator'

for i in range(len(get_date_tasks('alt_usher'))):
  print(i)
# get_eligible('song_leader')

0
1
2
3


In [4]:
assignments_file = 'data/previous-assignments-ipynb.csv'

# Initialize or read previous assignments
if os.path.exists(assignments_file):
    previous_assignments_df = pd.read_csv(assignments_file, index_col=0)
else:
    previous_assignments_df = pd.DataFrame(0, index=people, columns=tasks)
    previous_assignments_df['Rounds'] = 0
    
previous_assignments_df['Rounds'] += 1

# all_date_tasks
# tasks

In [5]:
# Calculate the average assignment frequency for each task
avg_assignments = pd.DataFrame(index=people, columns=tasks)
for person in people:
    for task in tasks:
        avg_assignments.loc[person, task] = previous_assignments_df.loc[person, task] / max(previous_assignments_df.loc[person, 'Rounds'], 1)

# Problem / Objective

In [6]:
# Create the LP problem
# We want to choose asignees so as to Maximize the deviation between their historical mean and the ideal mean
# Over time, we should converge to everyone having the ideal mean
prob = LpProblem("Task_Assignment", LpMaximize)

# Define decision variables

# assignments 
x = LpVariable.dicts("assign", ((person, task) for person in people for task in all_date_tasks), cat='Binary')

###
# Objective function: maximize the each persons deviation from the ideal average for each task
###

# todo make trim tasks names
prob += lpSum(
    (ideal_avg[trim_task_name(date_task)] - avg_assignments.loc[person, trim_task_name(date_task)]) 
    * x[(person, date_task)] 
    for person in people for date_task in all_date_tasks if is_eligible(person, trim_task_name(date_task))
)

# Constraints

In [7]:
# Only assign eligible people
for person in people:
    for task in tasks:
        if not is_eligible(person, task):
            for ineligible_task in get_date_tasks(task):
                prob += x[(person, ineligible_task)] == 0

# Do not assign a person to two excluded tasks in the same period
for task1 in tasks:
    for task2 in tasks:
        if is_excluded(task1, task2):
            for person in people:
                if is_eligible(person, task1) and is_eligible(person, task2):
                    for ineligible_pair in zip_longest(
                        get_date_tasks(task1), get_date_tasks(task2), fillvalue=0
                    ):
                        if (
                            0 not in ineligible_pair
                            and ineligible_pair[0] != ineligible_pair[1]
                        ):
                            for person in people:
                                prob += (
                                    x[(person, ineligible_pair[0])]
                                    + x[(person, ineligible_pair[1])]
                                    <= 1
                                )

# do not schedule the same person for the same task in the same month before pool exhausted
for person in people:
    for task in tasks:
        num_eligible = len(get_eligible(task))
        date_tasks = get_date_tasks(task)

        if num_eligible >= len(date_tasks):
            # we have an abundance everyone should go at least once
            prob += lpSum(x[(person, date_task)] for date_task in date_tasks) <= 1
        else:
            # Some may repeat, but no one should repeat more than one more than the other people
            prob += (
                lpSum(x[(person, date_task)] for date_task in date_tasks)
                <= (len(date_tasks) + num_eligible - 1) / num_eligible
            )

# Task limit constraints
# Task assignment constraints: each task is assigned to exactly one person
for task in all_date_tasks:
    prob += lpSum(x[(person, task)] for person in people) == 1


In [8]:
# Solve the problem
result = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4z/l87fr4g16qb0rxtm42zcxtym0000gn/T/89edc4cac57047bc98c9d5f254f4f183-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/4z/l87fr4g16qb0rxtm42zcxtym0000gn/T/89edc4cac57047bc98c9d5f254f4f183-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 303611 COLUMNS
At line 922917 RHS
At line 1226524 BOUNDS
At line 1230174 ENDATA
Problem MODEL has 303606 rows, 3649 columns and 610402 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -9.26384 - 0.30 seconds
Cgl0002I 2044 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 2304 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2399 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1771 strength

In [9]:
# Output the results
assignment = {}
for person in people:
    assigned_tasks = [task for task in all_date_tasks if x[(person, task)].varValue == 1]
    if assigned_tasks:
        assignment[person] = assigned_tasks
        for task in assigned_tasks:
            if roster.eligibility_df.loc[person, trim_task_name(task)] != 1:
                print(f"{person} is NOT eligilbe for {task}")
                
# assignment

# Output

In [10]:
# Output the results
assignments = pd.DataFrame(0, index=people, columns=all_date_tasks)
for person in people:
    for task in all_date_tasks:
        assignments.loc[person, task] = 1 if x[(person, task)].varValue == 1.0 else 0

# Update the previous assignments with the current ones
for person in people:
    for task in all_date_tasks:
        if assignments.loc[person, task]:
            previous_assignments_df.loc[person, trim_task_name(task)] += 1

# Save the updated assignments to a CSV file
previous_assignments_df.to_csv(assignments_file)

In [18]:
schedule_assignments = OrderedDict()
for person in people:
    if person in assignment:
        for task in assignment[person]:
            schedule_assignments[task] = person

schedule_entries = []
for key, value in list(schedule_assignments.items()):
    schedule_entries.append(f"{key}: {value}")

expected_tasks = set(all_date_tasks)
for etask in expected_tasks:
    if etask not in schedule_assignments.keys():
        print(f"{etask} MISSING!")

schedule.set_assignments(schedule_assignments)

display(HTML(render_schedule_to_html(schedule)))

1st Service 9:00,,4,,11,,18,,25
Song Leader,,"Nunn, Sam",,"Glover, Kole",,"Tipton, Dayle",,"Evans, Barret"
Opening Prayer,,"Scott, Rusty",,"Smiley, Justin",,"Nunn, Sam",,"Glover, Kole"
Lesson,,"Purcell, Lance",,"Kendrick, Mark",,"Ledbetter, Patrick",,"Tipton, Dayle"
Announcements,,"Nunn, Jeff",,"McAlister, Brady",,"Ledbetter, Patrick",,"Nunn, Jeff"
Song Leader,,"Kendrick, Mark",,"McAlister, Benson",,"Earp, Wyatt",,"Scott, Rusty"
Scripture Reading,,"McAlister, Grady",,"Scott, Rusty",,"Tipton, Sam",,"Purcell, Tripp"
Opening Prayer,,"Hight, Ben",,"Stroik, Darrin",,"Tipton, Dayle",,"Kendrick, Mark"
Closing Prayer,,"Taylor, Keith",,"Evans, Barret",,"Vinson, Joe",,"Purcell, Lance"
Table Aid Cup (N),,"Evans, Barret",,"Taylor, Keith",,"Holt, Jeff",,"Tipton, Sam"
Table Lead Cup (N),,"Glover, Kole",,"Holt, Jeff",,"Warren, David",,"Tipton, Dayle"


In [25]:
from pyquery import PyQuery as pq

doc = pq(filename='/Users/stipton/Desktop/congreation-roster/output/html/roster-8-2024.pdf.html')

cells = doc("td.duty-cell")
assignments = {}

for i in range(len(cells)):
  date_task = cells.eq(i).attr('data-duty')
  man = cells.eq(i).find("input").attr('value')
  assignments[date_task] = man

assignments

{'first_song_leader-4': 'Byers, Austin',
 'first_song_leader-11': 'Tipton, Dayle',
 'first_song_leader-18': 'Earp, Wyatt',
 'first_song_leader-25': 'Glover, Kole',
 'first_opening_prayer-4': 'McAnear, Walker',
 'first_opening_prayer-11': 'Dow, Benjamin',
 'first_opening_prayer-18': 'Holt, Jeff',
 'first_opening_prayer-25': 'McAlister, Benson',
 'first_lesson-4': 'Perez, Fred',
 'first_lesson-11': 'Tipton, Sam',
 'first_lesson-18': 'McAlister, Benson',
 'first_lesson-25': 'McAlister, Brady',
 'announcements-4': 'McAlister, Brady',
 'announcements-11': 'McAlister, Brady',
 'announcements-18': 'Nunn, Jeff',
 'announcements-25': 'Nunn, Jeff',
 'song_leader-4': 'Dow, Benjamin',
 'song_leader-11': 'Evans, Barret',
 'song_leader-18': 'Schiffman, Bobby',
 'song_leader-25': 'McAlister, Brady',
 'scripture_reading-4': 'McAnear, Jaxan',
 'scripture_reading-11': 'Addy, Levi',
 'scripture_reading-18': 'Ledbetter, Patrick',
 'scripture_reading-25': 'Purcell, Lance',
 'opening_prayer-4': 'McAlister, 